In [155]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import seaborn as sns

from gpx_parser import read_gpx
from analiser import (
    create_track_dataframe,
    add_angles_betwen_segments,
    add_vector_segments,
    add_arc_lengths_betwen_segments,
    add_arc_distances_from_start,
    add_distances_from_start,
    add_lengths_betwen_segments,
    add_oriented_angles_betwen_segments,
    add_velocities,
    make_plot,
    make_pairplt,
)

In [156]:
# df = read_gpx("Korotkov_2023", ["Gvandra"])

df = read_gpx("Chashnikova_2024_row", ["Трек_КОЛЬЦО12"])

# df = read_gpx("bad_example", ["track"])

In [157]:
add_arc_distances_from_start(df)
add_angles_betwen_segments(df)
add_lengths_betwen_segments(df)
# add_arc_lengths_betwen_segments(df)

p:\HikingSoft\GPXCleaner\spheric_geometry.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  B = (cos_b - cos_a * cos_c) / sin_a / sin_c
p:\HikingSoft\GPXCleaner\analiser.py:85: RuntimeWarning: invalid value encountered in scalar power
  return ((x[0] - y[0]) ** 2 + (x[1] - y[1])) ** 0.5


In [158]:
lons = np.array(df["rel_lon"])
lats = np.array(df["rel_lat"])
points = np.array(list(zip(lats, lons)))

In [159]:
epsilon = df["lengths_betwen_segments"].mean()
db = DBSCAN(eps=epsilon, min_samples=2).fit(points)
labels = db.labels_

In [160]:
with open('tmp.txt', 'w') as file:
    for point, label in zip(points, labels):
        file.write(f'{point[0]},{point[1]},{label}\n')

In [161]:
df['cluster_label'] = pd.Series(labels)

In [162]:
n_rolling_window = 20 # Размер скользящего окна
df[f'metric_{n_rolling_window}'] = df['angles_betwen_segments'].rolling(window=n_rolling_window).mean()

In [163]:
mini_df = df[['arc_distances_from_start', 'metric_20', 'cluster_label']]


# sns.pairplot(data=mini_df, hue='cluster_label', height=10, palette='Paired')
# plt.show()

In [164]:
threashold = 1.25

bad_labels = df[df['metric_20'] < threashold]['cluster_label'].values

clean_df = df[~df['cluster_label'].isin(bad_labels)]

In [165]:
import xml.etree.ElementTree as et

def create_gpx_file(coordinates, output_file):
    gpx = et.Element("gpx", version="1.1", xmlns="http://www.topografix.com/GPX/1/1")
    track = et.SubElement(gpx, 'trk')
    
    name = et.SubElement(gpx, 'name')
    name.text = 'tmp_name'

    track_segment = et.SubElement(track, 'trkseg')
    
    for lat, lon in coordinates:
        point = et.SubElement(track_segment, 'trkpt', attrib={'lat': str(lat), 'lon': str(lon)})

    tree = et.ElementTree(gpx)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)


In [166]:
lons = np.array(clean_df["lon"])
lats = np.array(clean_df["lat"])
points = np.array(list(zip(lats, lons)))

points = points * 180 / np.pi

In [167]:
create_gpx_file(points, 'tmp.gpx')